In [1]:
import pandas as pd
from pycaret.datasets import get_data
from pycaret.classification import *
import warnings
warnings.filterwarnings('ignore')

In [2]:
dataset = pd.read_csv('Back_Test_Result.csv')
dataset.shape

(1000, 10)

In [3]:
dataset.head()

,Candle_No,Current_Market_Fit,Current_Market,Current_Market_Stoploss,Rec1,Rec1_P,Rec2,Rec2_P,Rec3,Rec3_P
0,662.0,33.0,Bearish,0.0,4.0,BUY,3.0,BUY,80.0,BUY
1,5870.0,67.0,Bearish,0.0,75.0,BUY,29.0,SELL,11.0,SELL
2,4153.0,92.0,Bearish,0.0,22.0,BUY,3.0,SELL,0.0,BUY
3,158.0,33.0,Bearish,0.0,0.0,BUY,26.0,BUY,5.0,SELL
4,4013.0,0.0,Bullish,0.0,74.0,BUY,45.0,BUY,69.0,BUY


## One-Hot Encoding

In [4]:
dataset['Current_Market'] = dataset['Current_Market'].replace(['Bullish','Bearish'],[1,0])
dataset['Rec1_P'] = dataset['Rec1_P'].replace(['BUY','SELL'],[1,-1])
dataset['Rec2_P'] = dataset['Rec2_P'].replace(['BUY','SELL'],[1,-1])
dataset['Rec3_P'] = dataset['Rec3_P'].replace(['BUY','SELL'],[1,-1])

In [5]:
dataset['Rec1'] = dataset['Rec1'] * dataset['Rec1_P']
dataset['Rec2'] = dataset['Rec2'] * dataset['Rec2_P']
dataset['Rec3'] = dataset['Rec3'] * dataset['Rec3_P']

In [6]:
dataset = dataset.drop([
    'Candle_No',
    'Current_Market_Fit',
    'Current_Market_Stoploss',
#    'Rec1',
    'Rec1_P',
#    'Rec2',
    'Rec2_P',
#    'Rec3',
    'Rec3_P'
], axis=1)

In [7]:
dataset.head()

,Current_Market,Rec1,Rec2,Rec3
0,0,4.0,3.0,80.0
1,0,75.0,-29.0,-11.0
2,0,22.0,-3.0,0.0
3,0,0.0,26.0,-5.0
4,1,74.0,45.0,69.0


In [8]:
data = dataset.sample(frac=0.90, random_state=765).reset_index(drop=True)
data_unseen = dataset.drop(data.index).reset_index(drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (900, 4)
Unseen Data For Predictions: (100, 4)


In [9]:
exp_clf101 = setup(data = data, target = 'Current_Market', session_id=123, silent=True) 

,Description,Value
0,session_id,123
1,Target,Current_Market
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(900, 4)"
5,Missing Values,False
6,Numeric Features,3
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
compare_models()

IntProgress(value=0, description='Processing: ', max=69)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,15:47:22
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Extra Trees Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.5375,0.5673,0.5341,0.5365,0.5322,0.0756,0.0767,0.1600
dt,Decision Tree Classifier,0.5359,0.5358,0.5306,0.5391,0.5312,0.0717,0.0726,0.1130
gbc,Gradient Boosting Classifier,0.5294,0.5526,0.5501,0.5231,0.5337,0.0599,0.0610,0.0130
ada,Ada Boost Classifier,0.5230,0.5366,0.5150,0.5200,0.5148,0.0469,0.0472,0.0130
qda,Quadratic Discriminant Analysis,0.5135,0.5253,0.5524,0.5126,0.5298,0.0276,0.0278,0.0060
knn,K Neighbors Classifier,0.5008,0.4905,0.4861,0.4993,0.4894,0.0017,0.0019,0.1180
svm,SVM - Linear Kernel,0.4865,0.0000,0.4603,0.4839,0.4706,-0.0267,-0.0271,0.0050
lr,Logistic Regression,0.4784,0.4738,0.4249,0.4775,0.4476,-0.0439,-0.0433,0.2070
ridge,Ridge Classifier,0.4784,0.0000,0.4249,0.4775,0.4476,-0.0439,-0.0433,0.0040
lda,Linear Discriminant Analysis,0.4784,0.4739,0.4249,0.4775,0.4476,-0.0439,-0.0433,0.0030


/Users/yekrangian/Data/Forexience/Forexience/venv/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute standard_coef_ was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn(msg, category=FutureWarning)
/Users/yekrangian/Data/Forexience/Forexience/venv/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute standard_intercept_ was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn(msg, category=FutureWarning)
/Users/yekrangian/Data/Forexience/Forexience/venv/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute average_coef_ was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn(msg, category=FutureWarning)
/Users/yekrangian/Data/Forexience/Forexience/venv/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute average_intercept_ was deprecated in version 0.23 and will be removed in 0.25.
  warning

In [ ]:
MODEL = create_model('lightgbm')

In [ ]:
TUNED_MODEL = tune_model(MODEL)

In [ ]:
plot_model(TUNED_MODEL, plot = 'auc')

In [ ]:
plot_model(TUNED_MODEL, plot = 'pr')

In [ ]:
plot_model(TUNED_MODEL, plot='feature')

In [ ]:
plot_model(TUNED_MODEL, plot = 'confusion_matrix')

In [ ]:
predict_model(TUNED_MODEL);

In [ ]:
FINAL_MODEL = finalize_model(TUNED_MODEL)

In [ ]:
print(FINAL_MODEL)

In [ ]:
predict_model(FINAL_MODEL);

In [ ]:
unseen_predictions = predict_model(FINAL_MODEL, data=data_unseen)
unseen_predictions.head()

In [ ]:
save_model(FINAL_MODEL,'Final Model 04Nov2021')

In [ ]:
SAVED_FINAL_MODEL = load_model('Final Model 04Nov2021')

In [ ]:
new_prediction = predict_model(SAVED_FINAL_MODEL, data=data_unseen)

In [ ]:
new_prediction.head(20)

In [ ]:
final = new_prediction[['Current_Market','Label']]

In [ ]:
final.head(30)